In [1]:
pip install nltk

     |████████████████████████████████| 1.4MB 9.8MB/s eta 0:00:01
     |████████████████████████████████| 307kB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 675kB 30.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 22.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize, pos_tag, pos_tag_sents

### Import modules needed

In [3]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
# Create WordNetLemmatizer object 

sid = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Customized POS Tagger

In [4]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

In [5]:
df = pd.read_csv('opinion_survey.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
test_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,i can,i can,i can,i can,i can,i can,i can,i can,i can,i can,...,i can,i can,i can,i can,i can,i can,i can,i can,i can,-
1,i have no problems in log in,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,...,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem
2,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,...,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,"Yes, I have no problems with it"
3,i dont have problem in my canvas,i can easily login my canvas app in my mobile app,I can access my Canvas account using the avail...,i dont have problem in internet connection,i dont have problem with it,i dont have problem with it,i dont have problem with it,i dont have problem with it,i d

In [6]:
stoplist = stopwords.words('english') + ['though']
def remove_stopWords(w): 
    w = ' '.join(word for word in w.split() if word not in stoplist)
    return w

In [7]:
col_range = len(test_df.columns) # number of columns

for i in range(0,col_range):
    col = test_df.columns[i] # The current column
    test_df.loc[:,col] = test_df[col].apply(lambda x : str.lower(str(x))) ## To Lower Case
    test_df.loc[:,col] = test_df[col].apply(lambda x : " ".join(re.findall('[\w]+',x))) # Remove Punctuations
    test_df.loc[:,col] = test_df[col].apply(lambda x : remove_stopWords(x)) # Remove Stop words
    
    ##POS TAGGING
    texts = test_df.loc[:,col].tolist()
    tagged_texts = pos_tag_sents(map(word_tokenize, texts)) ### Tag every word in a row with POS
    
    ### Lemmatization
    new = []
    for i in tagged_texts:
        #if len(i) > 0:
        lemmatized_sentence = []
        for word, tag in i:
            tag = pos_tagger(tag) ### Convert POS Tag to known POS for simplification
            if tag is None: 
    # if there is no available tag, append the token as is 
                lemmatized_sentence.append(word) 
            else:         
    # else use the tag to lemmatize the token 
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag)) 

        lemmatized_sentence = " ".join(lemmatized_sentence) 
        #print(lemmatized_sentence)
        new.append(lemmatized_sentence)
    else:
        pass
    

    test_df['POS'] = new ## Store tagged words
    
test_df = test_df.replace(r'^\s*$', "neutral", regex=True) ## If row value is null, replace with neutral string
test_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.),POS
0,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,problems log,problem,problem,problem,problem,problem,problem,problem,problem,problem,...,problem,problem,problem,problem,problem,problem,problem,problem,problem,problem
2,problems,problems,problems,problems,problems,problems,problems,problems,problems,problems,...,problems,problems,problems,problems,problems,problems,problems,problems,yes problems,yes problem
3,dont problem canvas,easily login canvas app mobile app,access canvas account using available computer...,dont problem internet connection,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,...,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem
4,problems,access course,problem accessing canvas,dont issue,problem,problem,understand,easy recognize,manage,manage,...,im interested,course materials helped,yes teach us,yes demonstrate,encourage,communicate easy,problem,problem,problem,problem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,problem,problem,yes access,sometimes takes time load,yes people office always ready help,problem,yes clearly,yes well organized,problem,problem,...,interesting,problem,yes knows,yes teaching well,yes forums need answer,problem,yes,problem,yes sometimes extends dead

In [8]:
test_df.to_csv('lemmatized_opinion.csv', index=False)

### Sample POS Tags from first row

In [9]:
from pprint import pprint 

pprint(tagged_texts[:20])

[[],
 [('problem', 'NN')],
 [('yes', 'RB'), ('problems', 'NNS')],
 [('dont', 'NN'), ('problem', 'NN')],
 [('problem', 'NN')],
 [('yes', 'NNS'), ('course', 'NN')],
 [('give', 'VB'), ('week', 'NN'), ('deadline', 'NN'), ('activity', 'NN')],
 [('deadline', 'NN'),
  ('every', 'DT'),
  ('activities', 'NNS'),
  ('able', 'JJ'),
  ('us', 'PRP'),
  ('late', 'JJ')],
 [('sometimes', 'RB'),
  ('know', 'VBP'),
  ('due', 'JJ'),
  ('date', 'NN'),
  ('finish', 'JJ'),
  ('time', 'NN')],
 [('yes', 'RB'), ('given', 'VBN'), ('enough', 'JJ'), ('time', 'NN')],
 [('yes', 'RB'), ('especially', 'RB'), ('weekends', 'NNS')],
 [('give', 'VB'),
  ('enough', 'JJ'),
  ('time', 'NN'),
  ('accomplish', 'JJ'),
  ('activities', 'NNS'),
  ('canvas', 'VBP')],
 [('takes', 'VBZ'),
  ('time', 'NN'),
  ('week', 'NN'),
  ('submit', 'NN'),
  ('activities', 'NNS')],
 [('yes', 'NNS')],
 [('give', 'VB'),
  ('enough', 'JJ'),
  ('time', 'NN'),
  ('accomplish', 'JJ'),
  ('activities', 'NNS'),
  ('canvas', 'VBP')],
 [('give', 'VB'),
  

In [10]:
lem_df = pd.read_csv('lemmatized_opinion.csv')
lem_df = lem_df.iloc[0:, 0:-1] # Remove last columns
lem_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,problems log,problem,problem,problem,problem,problem,problem,problem,problem,problem,...,problem,problem,problem,problem,problem,problem,problem,problem,problem,problem
2,problems,problems,problems,problems,problems,problems,problems,problems,problems,problems,...,problems,problems,problems,problems,problems,problems,problems,problems,problems,yes problems
3,dont problem canvas,easily login canvas app mobile app,access canvas account using available computer...,dont problem internet connection,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,...,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem
4,problems,access course,problem accessing canvas,dont issue,problem,problem,understand,easy recognize,manage,manage,...,understand lesson,im interested,course materials helped,yes teach us,yes demonstrate,encourage,communicate easy,problem,problem,problem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,problem,problem,yes access,sometimes takes time load,yes people office always ready help,problem,yes clearly,yes well

In [11]:
comp = []
col_range = len(lem_df.columns) # number of columns

for i in range(0,col_range):
    col = test_df.columns[i] # The current column
    test_df['scores'] = test_df[col].apply(lambda x: sid.polarity_scores(x)) ## Get polarity score of every Column
    compound = test_df['scores'].apply(lambda score_dict: score_dict['compound']) ## Extract the compound from the results
    test_df = test_df.drop('scores', 1) # Drop score DF in every iteration
    compound = sum(compound)/140 # Get the mean compound of each columns
    comp.append(compound) # Save mean and append to list

In [12]:
from pprint import pprint

pprint(comp)

[0.17137571428571427,
 0.16313071428571432,
 0.15920571428571434,
 0.08488142857142857,
 0.2711971428571431,
 0.17636500000000008,
 0.26615285714285725,
 0.27341714285714314,
 0.31624142857142856,
 0.25183285714285736,
 0.20647571428571448,
 0.33951142857142835,
 0.37102928571428556,
 0.2830335714285715,
 0.24497928571428587,
 0.2714250000000001,
 0.2912928571428571,
 0.2971407142857143,
 0.2308800000000002,
 0.206052142857143,
 0.22575571428571445]


## Correlations

In [13]:
survey_df = pd.read_csv('survey_numerical_value.csv')
survey_df = survey_df.drop(survey_df.columns[0], axis=1)
survey_df = survey_df.drop(survey_df.index[[14,15,16,21,22,25,26]])
survey_df["Sentiment Score"] = comp



In [14]:
df1 = pd.read_csv('survey_numerical_value.csv')
df1 = df1.drop(df1.columns[0], axis=1)

merged = survey_df.combine_first(df1)

order = ['index','Strongly Agree', 'Agree', 'Disagree','Strongly Disagree','Numerical Value', 'Sentiment Score']
merged = merged.reindex(columns=order)
merged

,index,Strongly Agree,Agree,Disagree,Strongly Disagree,Numerical Value,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,124.0,12.0,1.0,3.0,3.84,0.171376
1,I can access my course using the mobile Canvas...,112.0,17.0,5.0,6.0,3.68,0.163131
2,I can access my Canvas account using the avail...,124.0,11.0,2.0,3.0,3.83,0.159206
3,I can easily get help when I have problems wit...,103.0,18.0,8.0,11.0,3.52,0.084881
4,I have no issues accessing my Canvas account u...,114.0,12.0,8.0,6.0,3.67,0.271197
5,I can access my Canvas account outside JRU.,122.0,11.0,6.0,1.0,3.81,0.176365
6,I can clearly understand the lessons on my Can...,105.0,21.0,9.0,5.0,3.61,0.266153
7,I recognize that the modules in my course are ...,116.0,19.0,3.0,2.0,3.78,0.273417
8,I can easily manage the navigation of the cour...,121.0,16.0,0.0,3.0,3.82,0.316241
9,I can easily manage the navigation of the cour...,103.0,18.0,10.0,9.0,3.54,0.251833


In [15]:
lem_df = merged.drop(merged.columns[[1,2,3,4]],axis=1)
lem_df

,index,Numerical Value,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,3.84,0.171376
1,I can access my course using the mobile Canvas...,3.68,0.163131
2,I can access my Canvas account using the avail...,3.83,0.159206
3,I can easily get help when I have problems wit...,3.52,0.084881
4,I have no issues accessing my Canvas account u...,3.67,0.271197
5,I can access my Canvas account outside JRU.,3.81,0.176365
6,I can clearly understand the lessons on my Can...,3.61,0.266153
7,I recognize that the modules in my course are ...,3.78,0.273417
8,I can easily manage the navigation of the cour...,3.82,0.316241
9,I can easily manage the navigation of the cour...,3.54,0.251833


In [16]:
#Aggregate Same Likert and Opinions first
# Manually fix inequal rows
'''
Numerical_mean  = corr.iloc[0:, 1].mean()
Sentiment_mean  = corr.iloc[0:, 2].mean()
means = [Numerical_mean,Sentiment_mean]
corr.loc[21] = ['Average'] + means
'''
                        ###Row  #Col
numerical_mean_first  = lem_df.iloc[13:17, 1].mean()
sentiment_mean_first = lem_df.iloc[13,2]

numerical_mean_sec  = lem_df.iloc[20:23, 1].mean()
sentiment_mean_sec = lem_df.iloc[20,2]

numerical_mean_third  = lem_df.iloc[24:27, 1].mean()
sentiment_mean_third = lem_df.iloc[24,2]

means1 = [numerical_mean_first,sentiment_mean_first]
lem_df.loc[29] = ['1st aggregation'] + means1

means2 = [numerical_mean_sec,sentiment_mean_sec]
lem_df.loc[30] = ['2nd aggregation'] + means2

means3 = [numerical_mean_third,sentiment_mean_third]
lem_df.loc[31] = ['3rd aggregation'] + means3

Numerical_mean  = lem_df.iloc[0:, 1].mean()
Sentiment_mean  = lem_df.iloc[0:, 2].mean()
means = [Numerical_mean,Sentiment_mean]
lem_df.loc[32] = ['Average'] + means

fixed_df = lem_df.drop([13,14,15,16,20,21,22,24,25,26])

fixed_df.reset_index(drop=True)
fixed_df.to_csv("fixed_ngram.csv", index=False)
fixed_df

,index,Numerical Value,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,3.840000,0.171376
1,I can access my course using the mobile Canvas...,3.680000,0.163131
2,I can access my Canvas account using the avail...,3.830000,0.159206
3,I can easily get help when I have problems wit...,3.520000,0.084881
4,I have no issues accessing my Canvas account u...,3.670000,0.271197
5,I can access my Canvas account outside JRU.,3.810000,0.176365
6,I can clearly understand the lessons on my Can...,3.610000,0.266153
7,I recognize that the modules in my course are ...,3.780000,0.273417
8,I can easily manage the navigation of the cour...,3.820000,0.316241
9,I can easily manage the navigation of the cour...,3.540000,0.251833


In [17]:
x_var = fixed_df.iloc[0:-1, 1]
y_var = fixed_df.iloc[0:-1, 2]
y_var

0     0.171376
1     0.163131
2     0.159206
3     0.084881
4     0.271197
5     0.176365
6     0.266153
7     0.273417
8     0.316241
9     0.251833
10    0.206476
11    0.339511
12    0.371029
17    0.244979
18    0.271425
19    0.291293
23    0.230880
27    0.225756
29    0.283034
30    0.297141
31    0.206052
Name: Sentiment Score, dtype: float64

In [18]:
import scipy

pearson_r = scipy.stats.pearsonr(x_var, y_var) #Pearson Correlation Analysis
pearson_r = np.reshape(pearson_r, (1, 2))
corr_analysis = pd.DataFrame(pearson_r, columns=["R", "p-Value"])
corr_analysis

,R,p-Value
0,0.08236,0.722655
